In [16]:
print(f"Lookup ES Aliase Mapping via REST API")

Lookup ES Aliase Mapping via REST API


In [17]:
!pip install elasticsearch==7.13.0

In [18]:
from elasticsearch import Elasticsearch
import os
import json
import pandas as pd
import jsondiff
import logging
from dotenv import load_dotenv
import socket
import requests
import warnings
warnings.filterwarnings("ignore")

In [19]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables 

True

In [20]:
''' It can be used to lookup old ES indices which are not mapped to any aliases for all smart suite env's '''
''' This is an API to lookup '''

' This is an API to lookup '

In [21]:
''' .env '''
# api_end_point='localhost'
# qa_env='QA-01|localhost:9200,QA-02|localhost:9200'
# prod_env='PROD-01|localhost:9200,PROD-02|localhost:9200'

' .env '

In [22]:
api_endpoint = os.getenv('api_end_point')

In [23]:
source_dict = {
    'local' : os.getenv('local_env'),
    'qa' : os.getenv('qa_env'),
    'prod' : os.getenv('prod_env'),
}

In [24]:
env = 'local'
# env = 'qa'
# env = 'prod'

In [25]:
host_lists_value = source_dict.get(env).split(",")
env_host_url = []
env_host_name = {}
for element in host_lists_value:
    env_name = element.split('|')[0]
    env_host_url_str = element.split('|')[1]
    env_host_url.append(env_host_url_str)
    env_host_name.update({env_host_url_str : env_name})

In [26]:
print(env_host_url)
print(env_host_name)

['localhost:9201']
{'localhost:9201': 'local-01'}


In [27]:
def get_request_api(host_lists):
    envs_name, envs, indices, docs, size = [], [], [], [], []
    try:
        for host in host_lists:
            print(f"env name : {env_host_name.get(host)}, host name : {host}")
            resp = requests.get(
                url='http://{}:8001/index/lookup_index_not_mapped_to_aliase?source_cluster=http://{}'.format(api_endpoint, host),
                timeout=5
            )
    
            if resp.status_code != 200:
                print(resp)
                return

            ''' request sucessfully '''
            # print(resp.json())

            for idx in list(resp.json()['unmapped_aliase'].keys()):
                envs_name.append(env_host_name.get(host))
                envs.append(host)
                indices.append(idx)
                docs.append(resp.json().get('unmapped_aliase').get(idx).get("doc_count"))
                size.append(resp.json().get('unmapped_aliase').get(idx).get("store.size"))

        df_unmapped_dict = {
            "environment" : envs_name,
            "host" : envs,
            "unmapped_index_name" : indices,
            "the number of docs" : docs,
            "store.size" : size
        }

        return df_unmapped_dict
            
    except Exception as e:
        print(e)

In [28]:
df_unmapped_dict = get_request_api(env_host_url)

env name : local-01, host name : localhost:9201
HTTPConnectionPool(host='localhost', port=8001): Max retries exceeded with url: /index/lookup_index_not_mapped_to_aliase?source_cluster=http://localhost:9201 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000246482A3AD0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


In [29]:
df = pd.DataFrame.from_dict(df_unmapped_dict)
# df.head(10)
display(df)

""


In [30]:
# Write DataFrame to CSV File with Default params.
df.to_csv(f"./[{env}]_df_unmapped_indices_all_envs.csv")